# Arctic Sea ice files 

* **Description**: RCP6.0 is missing year 2051, so just duplicate year 2050 so we'll get continuous timeseries
* **Input data**: MESACLIP daily timeseries data
* **Output data**: Netcdf with data
* **Creator**: Alice DuVivier
* **Date**: December 2025

### Import Packages

In [1]:
#import utils
import xarray as xr
import numpy as np
from glob import glob
import importlib
from datetime import datetime
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cmocean

import dask
from dask.distributed import Client
from dask_jobqueue import PBSCluster
from ncar_jobqueue import NCARCluster

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### ATM file

In [12]:
# set a dataset to read in to get grid information
varname = 'VBOT'

dir_in = '/glade/campaign/cgd/ppc/duvivier/arctic_actionable/DATA/mesaclip_all_arctic/daily/'+varname+'/'
file_in = 'MESACLIP_RCP6.0_007_2050_'+varname+'_alaska_daily.nc'
file_out = 'MESACLIP_RCP6.0_007_2051_'+varname+'_alaska_daily.nc'

ds = xr.open_mfdataset(dir_in+file_in, decode_times=False)

In [13]:
# look at time array
ds.time

<xarray.DataArray 'time' (time: 366)>
array([  0,   1,   2, ..., 363, 364, 365])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
Attributes:
    units:     days since 2049-12-31 00:00:00.000000
    calendar:  noleap

In [14]:
# adjust the units for time to shift forward one year
ds.time.attrs['units'] = 'days since 2050-12-31 00:00:00.000000'

In [15]:
# look at time array
ds.time

<xarray.DataArray 'time' (time: 366)>
array([  0,   1,   2, ..., 363, 364, 365])
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
Attributes:
    units:     days since 2050-12-31 00:00:00.000000
    calendar:  noleap

## Save the duplicated file

In [16]:
ds_out = ds
# add an attribute
ds_out.attrs['contents'] = f'Daily MESACLIP data for ocean point nearest Utqiagvik - 2051 is a duplicate of 2050 since the 2051 file is missing'

In [17]:
# Print the dimensions
print("Dimensions:")
for dim in ds_out.dims:
    print(f"\t{dim}: {ds_out[dim].values.shape}")

# Print the coordinates
print("Coordinates:")
for coord in ds_out.coords:
    print(f"\t{coord}:")
    print(f"\t\t{ds_out.coords[coord].values}")
    
# Print the attributes
print("Attributes:")
for attr in ds_out.attrs:
    print(f"\t{attr}: {ds_out.attrs[attr]}")
    

Dimensions:
	time: (366,)
	lev: (30,)
	ilev: (31,)
	cosp_prs: (7,)
	nbnd: (2,)
	cosp_tau: (7,)
	cosp_ht: (40,)
	cosp_sr: (15,)
	cosp_dbze: (15,)
	cosp_htmisr: (16,)
	cosp_tau_modis: (6,)
	cosp_scol: (10,)
	cosp_sza: (5,)
	lat: (160,)
	lon: (240,)
Coordinates:
	lev:
		[  3.64346569   7.59481965  14.35663225  24.61222     38.26829977
  54.59547974  72.01245055  87.82123029 103.31712663 121.54724076
 142.99403876 168.22507977 197.9080867  232.82861896 273.91081676
 322.24190235 379.10090387 445.9925741  524.68717471 609.77869481
 691.38943031 763.40448111 820.85836865 859.53476653 887.02024892
 912.64454694 936.19839847 957.48547954 976.32540739 992.55609512]
	ilev:
		[   2.25523952    5.03169186   10.15794743   18.55531707   30.66912293
   45.86747661   63.32348287   80.70141822   94.94104236  111.6932109
  131.40127063  154.58680689  181.86335266  213.95282075  251.70441717
  296.11721635  348.36658835  409.83521938  482.14992881  567.22442061
  652.33296901  730.44589162  796.36307061 

In [18]:
# calculate the size of the dataset in GB
size_gb = ds_out.nbytes / (1024**2)
print(f"The dataset is approximately {size_gb:.2f} MB.")

The dataset is approximately 108.21 MB.


In [19]:
fout = dir_in + file_out
print(fout)

/glade/campaign/cgd/ppc/duvivier/arctic_actionable/DATA/mesaclip_all_arctic/daily/VBOT/MESACLIP_RCP6.0_007_2051_VBOT_alaska_daily.nc


In [20]:
# Export the dataset to NetCDF with all attributes and coordinates
ds_out.to_netcdf(fout)